In [ ]:
using thesis, PRMaps
using Healpix
using Plots
using Statistics
using LsqFit

import Stripeline as Sl
import Pandas as Pd

In [ ]:
function run_scaling(
    tel_angles,
    sky_model::String,
    nside::Int
)
    
    strip = Pd.query(Pd.read_pickle("../instruments/lspe_strip_instrument.pkl"), :(frequency==43.0))
    signal = get_foreground_maps(strip, sky_model, nside)[1]

    camera = Sl.CameraAngles()
    obs_ideal, _ = makeIdealMapIQU(camera, signal, setup)
    
    obs_errored = PolarizedHealpixMap[]

    for i in tel_angles
        m, _ = makeErroredMapIQU(camera, i, signal, setup)
        push!(obs_errored, m)
    end

    error_i = [ (errored.i-obs_ideal.i) for errored in obs_errored]
    error_q = [ (errored.q-obs_ideal.q) for errored in obs_errored]
    error_u = [ (errored.u-obs_ideal.u) for errored in obs_errored]

    error_i_hist = [ i[isfinite.(i)] for i in error_i]
    error_q_hist = [ i[isfinite.(i)] for i in error_q]
    error_u_hist = [ i[isfinite.(i)] for i in error_u]

    std_error_i = [ std(i) for i in error_i_hist ]
    std_error_q = [ std(i) for i in error_q_hist ]
    std_error_u = [ std(i) for i in error_u_hist ]

    return (std_error_i, std_error_q, std_error_u)

end

In [ ]:
nside = 512
sky_model = "c1s0d0"
obs_days = 5
 
setup = PRMaps.Setup(
    sampling_freq_Hz = 50,
    total_time_s = 24. * 3600. * obs_days
    )
nothing

In [ ]:
errored_angles_deg = [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]
omega_wobble_deg = [0.0, 90.0, 180.0, 270.0]

xdata = log10.(range(0.01, stop=maximum(errored_angles_deg)+1, length=10000))

nothing

# Altitude offset angle

In [ ]:
tel_angs_altitude = []
for i in errored_angles_deg
    push!(tel_angs_altitude, Sl.TelescopeAngles(wheel2ang_0_rad = deg2rad(i)))
end
result_altitude = run_scaling(tel_angs_altitude, sky_model, nside)
nothing

L'errore è fittato con la seguente funzione:

$\log\epsilon = p_1 \log^2 x + p_2 \log x + p_3$

con x l'angolo di non idealità.

In [ ]:
@. model_linear(x,p) = p[1]*x^2 + p[2]*x + p[3]
log_error = log10.(result_altitude[2])
log_angle = log10.(errored_angles_deg)
fit_linear = curve_fit(model_linear, log_angle, log_error, [0.0, 0.5, 1.0])
print(fit_linear.param)


In [ ]:
scatter(log_angle, log_error, xlabel="Non ideality angle [log(degree)]", ylabel="std of the error [log(μK)]", label = "Simulated points")
plot!(xdata, model_linear(xdata, fit_linear.param), label = "Fit curve", title="Altitude angle θ")

# Ground offset angle

In [ ]:
tel_angs_ground = []
for i in errored_angles_deg
    push!(tel_angs_ground, Sl.TelescopeAngles(wheel3ang_0_rad = deg2rad(i)))
end
result_ground = run_scaling(tel_angs_ground, sky_model, nside)

In [ ]:
@. model_linear(x,p) = p[1]*x^2 + p[2]*x + p[3]
log_error = log10.(result_ground[1])
log_angle = log10.(errored_angles_deg)
fit_linear = curve_fit(model_linear, log_angle, log_error, [0.0, 0.5, 1.0])
print(fit_linear.param)

In [ ]:
scatter(log_angle, log_error, xlabel="Non ideality angle [log(degree)]", ylabel="std of the error [log(μK)]", label = "Simulated points")
plot!(xdata, model_linear(xdata, fit_linear.param), label = "Fit curve", title = "Ground angle φ")

# Fork angle

In [ ]:
tel_angs_fork = []
for i in errored_angles_deg
    push!(tel_angs_fork, Sl.TelescopeAngles(forkang_rad = deg2rad(i)))
end
result_fork = run_scaling(tel_angs_fork, sky_model, nside)

In [ ]:
@. model_linear(x,p) = p[1]*x^2 + p[2]*x + p[3]
log_error = log10.(result_fork[1])
log_angle = log10.(errored_angles_deg)
fit_linear = curve_fit(model_linear, log_angle, log_error, [0.0, 0.5, 1.0])
print(fit_linear.param)

In [ ]:
scatter(log_angle, log_error, xlabel="Non ideality angle [log(degree)]", ylabel="std of the error [log(μK)]", label = "Simulated points")
plot!(xdata, model_linear(xdata, fit_linear.param), label = "Fit curve", title = "Fork angle")

# Wobble angles

In [ ]:
tel_angs_0 = []
tel_angs_90 = []
tel_angs_180 = []
tel_angs_270 = []

for i in errored_angles_deg
    push!(tel_angs_0, Sl.TelescopeAngles(zVAXang_rad = deg2rad(i), ωVAXang_rad = deg2rad(0.0)))
    push!(tel_angs_90, Sl.TelescopeAngles(zVAXang_rad = deg2rad(i), ωVAXang_rad = deg2rad(90.0)))
    push!(tel_angs_180, Sl.TelescopeAngles(zVAXang_rad = deg2rad(i), ωVAXang_rad = deg2rad(180.0)))
    push!(tel_angs_270, Sl.TelescopeAngles(zVAXang_rad = deg2rad(i), ωVAXang_rad = deg2rad(270.0)))
end

result_0 = run_scaling(tel_angs_0, sky_model, nside)
result_90 = run_scaling(tel_angs_90, sky_model, nside)
result_180 = run_scaling(tel_angs_180, sky_model, nside)
result_270 = run_scaling(tel_angs_270, sky_model, nside)

In [ ]:
@. model_linear(x,p) = p[1]*x^2 + p[2]*x + p[3]
log_angle = log.(errored_angles_deg)

log_error_0 = log10.(result_0[1])
log_error_90 = log10.(result_90[1])
log_error_180 = log10.(result_180[1])
log_error_270 = log10.(result_270[1])

fit_linear_0 = curve_fit(model_linear, log_angle, log_error_0, [0.0, 1.0, 1.0])
fit_linear_90 = curve_fit(model_linear, log_angle, log_error_90, [0.0, 1.0, 1.0])
fit_linear_180 = curve_fit(model_linear, log_angle, log_error_180, [0.0, 1.0, 1.0])
fit_linear_270 = curve_fit(model_linear, log_angle, log_error_270, [0.0, 1.0, 1.0])

print(fit_linear_0.param)
print(fit_linear_90.param)
print(fit_linear_180.param)
print(fit_linear_270.param)

In [ ]:

a = scatter(log_angle, log_error_0, xlabel="Non ideality angle [log(degree)]", ylabel="std of the error [log(μK)]", label = "Simulated points ω=0°")
a = plot!(xdata, model_linear(xdata, fit_linear_0.param), label = "Fit curve")

b = scatter(log_angle, log_error_90, xlabel="Non ideality angle [log(degree)]", ylabel="std of the error [log(μK)]", label = "Simulated points ω=90°")
b = plot!(xdata, model_linear(xdata, fit_linear_90.param), label = "Fit curve")

c = scatter(log_angle, log_error_180, xlabel="Non ideality angle [log(degree)]", ylabel="std of the error [log(μK)]", label = "Simulated points ω=180°")
c = plot!(xdata, model_linear(xdata, fit_linear_180.param), label = "Fit curve")

d = scatter(log_angle, log_error_270, xlabel="Non ideality angle [log(degree)]", ylabel="std of the error [log(μK)]", label = "Simulated points ω=270°")
d = plot!(xdata, model_linear(xdata, fit_linear_270.param), label = "Fit curve")

plot(a,b,c,d, layaut = (4,1), size = (1200,600), margin = 6*Plots.mm, plot_title="Wobble angles")